In [8]:
import random
import wandb

# Start a new wandb run to track this script.
run = wandb.init(
    # Set the wandb entity where your project will be logged (generally your team name).
    entity="finnfreiheit",
    # Set the wandb project where this run will be logged.
    project="test_1",
    # Track hyperparameters and run metadata.
    config={
        "learning_rate": 0.02,
        "architecture": "CNN",
        "dataset": "CIFAR-100",
        "epochs": 10,
    },
)

# Simulate training.
epochs = 10
offset = random.random() / 5
for epoch in range(2, epochs):
    acc = 1 - 2**-epoch - random.random() / epoch - offset
    loss = 2**-epoch + random.random() / epoch + offset

    # Log metrics to wandb.
    run.log({"acc": acc, "loss": loss})

# Finish the run and upload any remaining data.
run.finish()

wandb: Currently logged in as: finnfrei (finnfreiheit) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: ERROR The nbformat package was not found. It is required to save notebook history.


acc,▁▄▅▅█▇██
loss,█▅▃▂▂▁▁▁
acc,0.77519
loss,0.23788


In [14]:
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

#from assessment import assesment_utils
#from assessment.assesment_utils import Classifier
#import utils

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.is_available()

False

In [15]:
import sys
sys.path.append("..")

from src.models import Classifier

lidar_cnn = Classifier(1).to(device)
for param in lidar_cnn.parameters():
    lidar_cnn.requires_grad = False
lidar_cnn.eval()


Classifier(
  (embedder): Sequential(
    (0): Conv2d(1, 50, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(50, 100, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(100, 200, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Conv2d(200, 200, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): ReLU()
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (12): Flatten(start_dim=1, end_dim=-1)
  )
  (classifier): Sequential(
    (0): Linear(in_features=3200, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=1, bias=True)
  )
)

In [16]:
from src.datasets import MyDataset

BATCH_SIZE = 32
VALID_BATCHES = 10
N = 9999

valid_N = VALID_BATCHES*BATCH_SIZE
train_N = N - valid_N

train_data = MyDataset("../data/assessment/", 0, train_N)
train_dataloader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)

valid_data = MyDataset("../data/assessment/", train_N, N)
valid_dataloader = DataLoader(valid_data, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)



In [17]:
from src.models import Embedder

img_embedder = Embedder(4).to(device)
lidar_embedder = Embedder(1).to(device)



In [18]:
from src.models import ContrastivePretraining

CILP_model = ContrastivePretraining(img_embedder=img_embedder, lidar_embedder=lidar_embedder).to(device)
optimizer = Adam(CILP_model.parameters(), lr=0.0001)
loss_img = nn.CrossEntropyLoss()
loss_lidar = nn.CrossEntropyLoss()
ground_truth = torch.arange(BATCH_SIZE, dtype=torch.long).to(device)
epochs = 3

In [19]:
def get_CILP_loss(batch):
    rbg_img, lidar_depth, class_idx = batch
    logits_per_img, logits_per_lidar = CILP_model(rbg_img, lidar_depth)
    total_loss = (loss_img(logits_per_img, ground_truth) + loss_lidar(logits_per_lidar, ground_truth))/2
    return total_loss, logits_per_img

In [20]:
from src.visualization import print_CILP_results

for epoch in range(epochs):
    CILP_model.train()
    train_loss = 0
    for step, batch in enumerate(train_dataloader):
        optimizer.zero_grad()
        loss, logits_per_img = get_CILP_loss(batch)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
    print_CILP_results(epoch, train_loss/step, logits_per_img, is_train=True)

    CILP_model.eval()
    valid_loss = 0
    for step, batch in enumerate(valid_dataloader):
        loss, logits_per_img = get_CILP_loss(batch)
        valid_loss += loss.item()
    print_CILP_results(epoch, valid_loss/step, logits_per_img, is_train=False)

Epoch 0
Train Loss: 3.085106645255144 
Similarity:
tensor([[0.9960, 0.2664, 0.7968,  ..., 0.5645, 0.4156, 0.3492],
        [0.2992, 0.9953, 0.2646,  ..., 0.2267, 0.9766, 0.2520],
        [0.7784, 0.3110, 0.9908,  ..., 0.6760, 0.2901, 0.2194],
        ...,
        [0.5929, 0.2448, 0.6733,  ..., 0.9945, 0.2106, 0.6167],
        [0.3771, 0.9698, 0.2555,  ..., 0.2325, 0.9939, 0.2596],
        [0.2953, 0.2855, 0.2259,  ..., 0.5776, 0.2670, 0.9963]],
       grad_fn=<DivBackward0>)
Valid Loss: 3.1927100106289514 
Similarity:
tensor([[0.9885, 0.8773, 0.4653,  ..., 0.5471, 0.5718, 0.9677],
        [0.9365, 0.9974, 0.3049,  ..., 0.2914, 0.3206, 0.9615],
        [0.4030, 0.2808, 0.9944,  ..., 0.5814, 0.5240, 0.3419],
        ...,
        [0.4802, 0.2921, 0.5523,  ..., 0.9991, 0.9933, 0.4463],
        [0.5170, 0.3323, 0.4893,  ..., 0.9937, 0.9984, 0.4879],
        [0.9944, 0.9426, 0.3629,  ..., 0.4923, 0.5222, 0.9962]],
       grad_fn=<DivBackward0>)
Epoch 1
Train Loss: 3.0299126904204514 
Similar

In [21]:
for param in CILP_model.parameters():
    CILP_model.requires_grad = False

In [22]:
projector = nn.Sequential(
    nn.Linear(200, 1000),   # input: 200-dim CLIP embedding
    nn.ReLU(),
    nn.Linear(1000, 500),
    nn.ReLU(),
    nn.Linear(500, 200 * 4 * 4)       # output: 3200-dim get_embs space
).to(device)

In [23]:
def get_projector_loss(model, batch):
    rbg_img, lidar_depth, class_idx = batch
    imb_embs = CILP_model.img_embedder(rbg_img)
    lidar_emb = lidar_cnn.get_embs(lidar_depth)
    pred_lidar_embs = model(imb_embs)
    return nn.MSELoss(pred_lidar_embs, lidar_emb)

In [25]:
from src.training import train_model

epochs = 40
optimizer = torch.optim.Adam(projector.parameters())
train_model(projector, optimizer, get_projector_loss, epochs, train_dataloader, valid_dataloader)

TypeError: get_projector_loss() missing 1 required positional argument: 'batch'

In [ ]:
from src.models import RGB2LiDARClassifier

my_classifier = RGB2LiDARClassifier(projector=projector, CILP_model=CILP_model, lidar_cnn=lidar_cnn)

In [24]:
def get_correct(output, y):
    zero_tensor = torch.tensor([0]).to(device)
    pred = torch.gt(output, zero_tensor)
    correct = pred.eq(y.view_as(pred)).sum().item()
    return correct

def get_valid_metrics():
    my_classifier.eval()
    correct = 0
    batch_correct = 0
    for step, batch in enumerate(valid_dataloader):
        rbg_img, _, class_idx = batch
        output = my_classifier(rbg_img)
        loss = nn.BCEWithLogitsLoss()(output, class_idx)
        batch_correct = get_correct(output, class_idx)
        correct += batch_correct
    print(f"Valid Loss: {loss.item():2.4f} | Accuracy {correct/valid_N:2.4f}")

get_valid_metrics()

NameError: name 'my_classifier' is not defined

In [ ]:
epochs = 5
optimizer = torch.optim.Adam(my_classifier.parameters())

my_classifier.train()
for epoch in range(epochs):
    correct = 0
    batch_correct = 0
    for step, batch in enumerate(train_dataloader):
        optimizer.zero_grad()
        rbg_img, _, class_idx = batch
        output = my_classifier(rbg_img)
        loss = nn.BCEWithLogitsLoss()(output, class_idx)
        batch_correct = get_correct(output, class_idx)
        correct += batch_correct
        loss.backward()
        optimizer.step()
    print(f"Train Loss: {loss.item():2.4f} | Accuracy {correct/train_N:2.4f}")
    get_valid_metrics()